In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from boostlets_mod import genBoostlet
import matplotlib.pyplot as plt


## Import image

In [ ]:
import os
from mod_RIRIS_func import load_sk, load_DB_ZEA, rand_downsamp_RIR, ImageOps, jitter_downsamp_RIR

# Tamaño diccionario // Tamaño imagen interpolada
M, N  = 128, 128
# Imagen tamaño:
M0, N0 = 100, 100
ratio_mics = 0.5
Tstart = 0
Tend = Tstart+M0

# ---------- LOAD Image ---------------------------
room= 'Balder'
folder = "./dependencies/measurementData"
file = room+"RIR.mat"
file_path = os.path.join(folder, file)

full_image = load_DB_ZEA(file_path)[0]
orig_image = full_image[Tstart:Tend, :N0]
mask0, _ = jitter_downsamp_RIR(orig_image.shape, ratio_t=1, ratio_x=ratio_mics)

# ----------------------------------------------------
# Extrapolation
# ----------------------------------------------------
extr_size = (M,N)
extrap_mode = 'pad'
imOps = ImageOps(orig_image.shape, mask=mask0, extrap_shape=extr_size, mode=extrap_mode) 

image = imOps.expand_image(orig_image)
mask = imOps.get_mask(image)

fig, ax = plt.subplots(1,2)
ax[0].imshow(orig_image[Tstart:Tstart+100, :])
ax[1].imshow((orig_image*mask0)[Tstart:Tstart+100, :])

ax[0].axis('off')
ax[1].axis('off') 
plt.show()

## Pareto con S=1 n_thetas=5. Diferentes beta_sets


In [ ]:
# ----------------------------------------------------
# Pareto
# ----------------------------------------------------
from mod_RIRIS_func import load_sk, computePareto, iffst, ista, linear_interpolation_fft

S, n_thetas = 1, 5
build_dict = dict(Sk_type = 'boostlet', N = N, S = S, n_thetas = n_thetas)
Sk1 = load_sk(folder='./', file='dummy', build_dict=build_dict)

beta_set = np.logspace(-2.5, -1, 50)
beta_star1, Jcurve = computePareto(image*mask, mask, Sk1, beta_set)

beta_set = np.logspace(-3, -0.5, 50)
beta_star2, Jcurve = computePareto(image*mask, mask, Sk1, beta_set)

beta_set = np.logspace(-5, -0.1, 50)
beta_star3, Jcurve = computePareto(image*mask, mask, Sk1, beta_set)


## ISTA con dict S=1 n_thetas=5. Diferentes beta_star

In [ ]:
print(f"Para dict con S={S}, n_thetas={n_thetas}")
print(f"Different beta_sets, dan:")
print("-------------------------------")
print(f"beta_star1 = {beta_star1}")
print(f"beta_star2 = {beta_star2}")
print(f"beta_star3 = {beta_star3}")


In [ ]:
# ----------------------------------------------------
# ISTA recovery
# ----------------------------------------------------
epsilon = 9.4e-6
alpha1 = ista(image*mask, mask, Sk1, beta=beta_star1, epsilon=epsilon, max_iterations=15 )

In [ ]:
alpha2 = ista(image*mask, mask, Sk1, beta=beta_star2, epsilon=epsilon, max_iterations=15 )

In [ ]:
alpha3 = ista(image*mask, mask, Sk1, beta=beta_star3, epsilon=epsilon, max_iterations=15 )

+ Para S=1, n_thetas=5 hay que manipular el beta_set pata obtener el maximo de J en computePareto.
+ Aun así, para estos parametros, los betas optimos (beta_star) obtenidos, hacen que ISTA converja

## Pareto e ISTA para S=2 n_thetas=5

In [ ]:
S, n_thetas = 2, 5
build_dict = dict(Sk_type = 'boostlet', N = N, S = S, n_thetas = n_thetas)
Sk2 = load_sk(folder='./', file='dummy', build_dict=build_dict)

beta_set = np.logspace(-2.5, -1, 50)
beta_star, Jcurve = computePareto(image*mask, mask, Sk2, beta_set)
print(f"Dict S={S}, n_thetas={n_thetas}")
print(f"beta_star = {beta_star}")

In [ ]:
beta_set = np.logspace(-3, -0.1, 50)
beta_star, Jcurve = computePareto(image*mask, mask, Sk2, beta_set)
print(f"Dict S={S}, n_thetas={n_thetas}")
print(f"beta_star = {beta_star}")

In [ ]:
# ----------------------------------------------------
# ISTA recovery
# ----------------------------------------------------
epsilon = 9.4e-6
alpha = ista(image*mask, mask, Sk2, beta=beta_star, epsilon=epsilon, max_iterations=15 )

## Pareto e ISTA para S=1, n_thetas = 20

In [ ]:
S, n_thetas = 1, 20
build_dict = dict(Sk_type = 'boostlet', N = N, S = S, n_thetas = n_thetas)
Sk1_20 = load_sk(folder='./', file='dummy', build_dict=build_dict)

beta_set = np.logspace(-5, -0.1, 50)
beta_star, Jcurve = computePareto(image*mask, mask, Sk1_20, beta_set)
print(f"Dict S={S}, n_thetas={n_thetas}")
print(f"beta_star = {beta_star}")

# ----------------------------------------------------
# ISTA recovery
# ----------------------------------------------------
epsilon = 9.4e-6
alpha = ista(image*mask, mask, Sk1_20, beta=beta_star, epsilon=epsilon, max_iterations=15 )

+ Para S=1, n_thetas=20 ISTA no converge tras calcular Pareto un beta_Star = 5e-5

In [ ]:
S, n_thetas = 3, 7
build_dict = dict(Sk_type = 'boostlet', N = N, S = S, n_thetas = n_thetas)
Sk1_20 = load_sk(folder='./', file='dummy', build_dict=build_dict)

# Sk = np.delete(Sk1_20, [30, 31, 29, 32], axis=2)
# Sk = np.delete(Sk1_20, [28, 29, 30, 31, 32, 33, 34], axis=2)
# Sk = Sk1_20[:,:, [0, 5,6,7,8,9,10,11,12,13,14]]
# Sk = Sk1_20
# Sk = np.delete(Sk1_20, [24, 25, 26], axis=2) # Guay
# Sk = np.delete(Sk1_20, [24, 25, 26, 31, 32, 33], axis=2) # Guay
Sk = np.delete(Sk1_20, [24, 25, 26, 31, 32, 33, 38, 39, 40], axis=2) # Guay




epsilon = 9.4e-6
# alpha = ista(image*mask, mask, Sk, beta=0.05, epsilon=epsilon, max_iterations=50 ) # No converge
# alpha = ista(image*mask, mask, Sk, beta=0.9, epsilon=epsilon, max_iterations=50 ) # converge
# alpha = ista(image*mask, mask, Sk, beta=0.05, epsilon=epsilon, max_iterations=50 ) # NO converge
# alpha = ista(image*mask, mask, Sk, beta=0.001, epsilon=epsilon, max_iterations=50 ) # NO converge
alpha = ista(image*mask, mask, Sk, beta=0.1, epsilon=epsilon, max_iterations=50 ) # converge



# recover inpainted image from sparse coefficients (Eq. 19)
image_recov = iffst(alpha, Sk)
final_image = imOps.recover_image(image_recov)

images = [orig_image[:100,:], (orig_image*mask0)[:100,:], final_image[:100,:]]
titles = ['Original Image', 'Masked Image', 'Final reconst image']

fig, ax = plt.subplots(1, len(images), figsize=(18, 6))
for i in range(len(images)):
    ax[i].imshow(images[i])
    ax[i].set_title(titles[i])
    ax[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:

images = [orig_image[:100,:], (orig_image*mask0)[:100,:], final_image[:100,:]]
titles = ['Original Image', 'Masked Image', 'Final reconst image']

fig, ax = plt.subplots(1, len(images), figsize=(18, 6))
for i in range(len(images)):
    ax[i].imshow(images[i])
    ax[i].set_title(titles[i])
    ax[i].axis('off')
plt.tight_layout()
plt.show()


+ Con otros beta como $\beta^* = 0.5$ o 1, parece que ISTA converge

## Pareto e ISTA para S=2,3, n_thetas = 20